# L-shaped example

Consider the following two-stage problem, consisting in the optimal capacity investment in various types of ice-cream production plants.
Four plants are considered and they can produce three different ice-cream flavors.
The demand in the next period has to be satisfied for each of the three flavors, and is equal to some random value $\xi$ for flavor 1, 3 for flavor 2, and 2 for flavor 3.
$\xi$ is discrete with three realizations 3, 5, 7, associated to the probabilities 0.3, 0.4 and 0.3 respectively.
There is a budget constraint on the investment and also a constraint on the minimum total capacity.
The minimum total capacity to be installed is 12 and we have a budget limit of 120.
The unit capacity cost for the four plants are 10, 7, 16 and 6, respectively.
The cost per production unit of plant $i$ for flavor $j$ is denoted $a_{ij}$ and gathering all the costs, we can construct the matrix
$$
A = \begin{pmatrix}
  40 & 24 & 4 \\
  45 & 27 & 4.5 \\
  32 & 19.2 & 3.2 \\
  55 & 33 & 5.5
\end{pmatrix}
$$
We aim to minimize the total cost (investment and production costs).
Formulate the problem and solve it using the L-Shaped method.
You have to provide the code for L-Shaped implementation and properly comment it.

In [ ]:
using JuMP
using LinearAlgebra
using Statistics

In [ ]:
using SparseArrays  # allows to benefit from sparse matrices

In [ ]:
using HiGHS

solver = HiGHS.Optimizer

We focus here on the code. More attention should be paid to the explanations.

## Problem Overview

Four new plants are considered and they can produce three different flavors of ice cream. We have to decide the capacity to install in each of these plants, by minimizing in the first stage the investment costs and minimizing the operating costs in the second stage, while satisfying the unknown demand.

The installed capacity is at least 12 and the investment cannot exceed 120.

The problem is
\begin{align*}
    \min_x & \sum_{i=1}^4 c_i x_i + E_ξ[Q(x,ξ)] \\
    \mbox{s.t. } & \sum_{i=1}^{4} x_i \geq 12 \\
                 & \sum_{i=1}^4 c_i x_i \leq 120 \\
                 & x \geq 0
\end{align*}
where
\begin{align*} 
Q(x,ξ) = \min_y\ & \sum_{i,j} A_{ij} y_{ij} \\
\mbox{s.t. } & \sum_{j=1}^3 y_{ij} \leq x_i,\ \forall\, i \\
             & \sum_{i=1}^4 y_{ij} \geq d_j(\xi),\ \forall\, j \\
             & y \geq 0
\end{align*}

## Extended form

We start by considering the extensive form in order to have a reference solution.

In [ ]:
openingCosts = [10, 7, 16, 6]
cost = [40 24 4; 45 27 4.5; 32 19.2 3.2; 55 33 5.5]

In [ ]:
m = Model()

(nFactories, nFlavors) = size(cost)

prob = [0.3 0.4 0.3]
nScenarios = length(prob)

#minCapacity = 12
minCapacity = 0   # we relax the minimum capacity constraint in order to produce a feasibility cut

maxInvestment = 120
scenarios = [3 5 7]
demands = [0 3 2]

@variable(m, x[1:nFactories] >= 0)
@variable(m, y[1:nScenarios, 1:nFactories, 1:nFlavors] >= 0)

@constraint(m, sum(x[i] for i in 1:nFactories) >= minCapacity)
@constraint(m, sum(openingCosts[i]*x[i] for i in 1:nFactories) <= maxInvestment)

maxProd = []

k = 0
for s = 1:nScenarios
    for j = 1:nFactories
        k += 1
        push!(maxProd, @constraint(m, sum(y[s,j,i] for i in 1:nFlavors) <= x[j]))
    end
end

for s = 1:nScenarios
    @constraint(m, sum(y[s,j,1] for j in 1:nFactories) >= scenarios[s])
    for d = 2:nFlavors
        @constraint(m, sum(y[s,j,d] for j in 1:nFactories) >= demands[d])
    end
end

@objective(m, Min, sum(openingCosts[i]*x[i] for i in 1:nFactories) +
    sum(prob[s]*sum(sum(cost[j,i]*y[s,j,i] for i in 1:nFlavors) for j = 1:nFactories) for s=1:nScenarios)
)

Let's print the complete model:

In [ ]:
println(m)

For instance, the constraint on maximum production can be read as

In [ ]:
maxProd[5]

We can solve the problem by setting the solve and calling the method optimize.

In [ ]:
set_optimizer(m, solver)
optimize!(m)

In [ ]:
value.(x)

In [ ]:
objective_value(m)

The solution is therefore $x^* = (8/3, 4, 13/3, 2)$.

## 2-stage formulation

We first create the first-stage problem.

In [ ]:
function firststage()
    m = Model(solver)

    @variable(m, x[1:nFactories] >= 0)
    @variable(m, θ)
    
    @constraint(m, sum(x[i] for i in 1:nFactories) >= minCapacity)
    @constraint(m, sum(openingCosts[i]*x[i] for i in 1:nFactories) <= maxInvestment)

    @objective(m, Min, sum(openingCosts[i]*x[i] for i in 1:nFactories))
    
    return m, x, θ
end

Note that we have not yet set $\theta$ in the objective as there is currently no constraint on $\theta$.

In [ ]:
m, x, θ = firststage()
println(m)

Once there is at least one constraint on $\theta$, we can ajust the master objective function.

In [ ]:
function master_objective(m::Model, x, θ)
    @objective(m, Min, sum(openingCosts[i]*x[i] for i in 1:nFactories) + θ)
    return m
end

We then create the second-stage problem.

We write all the programs in standard form.

We start by setting the core of the second stage.

In [ ]:
function secondstageCore(x, ξ)
    m = Model()

    @variable(m, y[1:nFactories, 1:nFlavors] >= 0)

    # T = sparse([-1 0 0 0; 0 -1 0 0; 0 0 -1 0; 0 0 0 -1; 0 0 0 0; 0 0 0 0; 0 0 0 0])
    # T = sparse(rows,cols,vals) 
    T = [ sparse([1,2,3,4], [1,2,3,4], [-1,-1,-1,-1]); spzeros(3,4) ]
#    T = [ -I; spzeros(3,4) ]

    h = sparse([ zeros(nFactories); ξ; demands[2:nFlavors]])

#    @constraintref recourseConstraints[1:(nFactories+nFlavors)]
    recourseConstraints = []

    return m, y, recourseConstraints, h, T
end 

The exact form of the second problem depends on the scenario.

In [ ]:
function secondstage(x, ξ)
    m, y, recourseConstraints, h, T = secondstageCore(x, ξ)

    # Capacity constraints
    for i = 1:nFactories
        push!(recourseConstraints, @constraint(m, sum(y[i,j] for j in 1:nFlavors) <= x[i]))
    end

    # Demand satisfaction constraints
    push!(recourseConstraints, @constraint(m, sum(y[j,1] for j in 1:nFactories) >= ξ))
    for d = 2:nFlavors
        push!(recourseConstraints, @constraint(m, sum(y[j,d] for j in 1:nFactories) >= demands[d]))
    end

    @objective(m, Min, sum(sum(cost[i,j]*y[i,j] for j in 1:nFlavors) for i = 1:nFactories))

    set_optimizer(m, solver)
    optimize!(m)
    
    return m, recourseConstraints, h, T
end

The second-stage feasibility problem to solve to obain the components of the feasibility cut can be built with the method below.

In [ ]:
function secondstagefeasibility(x, ξ)
    m, y, recourseConstraints, h, T = secondstageCore(x, ξ)

    t = nFactories+nFlavors
    
    @variable(m, w[1:t] >= 0)

    for i = 1:nFactories
        push!(recourseConstraints, @constraint(m, sum(y[i,j] for j in 1:nFlavors)-w[i] <= x[i]))
    end

    push!(recourseConstraints, @constraint(m, sum(y[j,1] for j in 1:nFactories)+w[nFactories+1] >= ξ))
    for d = 2:nFlavors
        push!(recourseConstraints, @constraint(m, sum(y[j,d] for j in 1:nFactories)+w[nFactories+d] >= demands[d]))
    end

    @objective(m, Min, sum(w[i] for i in 1:t))

    set_optimizer(m, solver)
    optimize!(m)
    σ = dual.(recourseConstraints)  # since the objective function is a minimization, we have the correct sign.

    return σ, h, T
end

We stop when $\theta >= \mathcal{Q}(x)$. However, this could fail due to numerical error, so we adjust the constraint with a predifined tolerance, and we defined a maximum number of iterations in order to avoid to go on an infinite loop if everything goes wrong.

In [ ]:
function stop(expected_Q, θ, k, tol = 1e-10)
    nmax = 200  # circuit breaker
    if ((θ >= expected_Q-tol) || (k == nmax))
        return true
    else
        return false
    end
end

JuMP is built on Math Optimization Interface (MOI). We take advantage of the associated flags.

The complete L-Shape algorithm is given below.

In [ ]:
function lshaped(scenarios, prob)
    nScenarios = length(scenarios)
    
    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = 0  # number of optimality cuts
    
    first, x, θ = firststage()
    n = length(x)

    Q = +Inf
    valθ = -Inf
    
    while (!stop(Q, valθ, k))
        k += 1

        println(first)

        optimize!(first)
        status = termination_status(first)
        
        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, x, first
        end
        
        # status == MOI.OPTIMAL
        xsol = value.(x)
        # println("xsol = ", xsol, "   ", "θ = ", value.(θ))
        E = zeros(n)'
        e = 0.0
        Q = 0.0
        
        # Solve the second-stage programs
        for i = 1:nScenarios
            p = prob[i]
            
            m, scstrs, h, T = secondstage(xsol, scenarios[i])
            status = termination_status(m)
            if (status == MOI.INFEASIBLE)
                if dual_status(m) == MOI.INFEASIBILITY_CERTIFICATE
                    # If the solver emits a infeasibility certificate, we can rely on it
                    # Note: this has been tested with HiGHS only
                    # Get dual ray from constraints (infeasibility certificate):
                    # For each constraint, try to extract its dual multiplier

                    # We cannot use all_constraints as JuMP reorganize the constraints by blocks of the same type
                    # σ = dual.(all_constraints(m; include_variable_in_set_constraints=false))
                    σ = dual.(scstrs)

                    # The sign of the dual variables depend of the sense of the optimization (min or max)
                    # Julia uses the conic duality (see https://jump.dev/MathOptInterface.jl/stable/background/duality/)
                    # As a consequence, we have to take the opposite of the dual variables in case of a minimization
                    sense = objective_sense(m)
                    if (sense == MOI.MAX_SENSE)
                        σ *= -1
                    end
                else
                    # No infeasibility certificate available
                    # We explictly solve the feasibility second-stage problem
                    σ, h, T = secondstagefeasibility(xsol, scenarios[i])
                end

                # Build a feasibility cut
                E = σ'*T
                @constraint(first, sum(E[i]*x[i] for i in 1:nFactories) >= σ'*h)
                nfcuts += 1
                break;
            elseif (status == MOI.OPTIMAL)
                Q += p*objective_value(m)
                # Build the optimality cut component
                π = dual.(scstrs)
                E += p*(π'*T)
                e += p*(dot(π,h))
            else
                println("Error second-stage resolution: status ", status)
                return status, x, first
            end
        end
        
        if (status == MOI.OPTIMAL)
            # add an optimality cut if it improves the lower bound
            if (nocuts == 0)
                # add θ in the problem
                @constraint(first, con, sum(E[i]*x[i] for i in 1:nFactories) + θ >= e)
                master_objective(first, x, θ)
            else
                valθ = value.(θ)
                if (valθ < Q)
                    @constraint(first, sum(E[i]*x[i] for i in 1:nFactories) + θ >= e)
                end
            end
            nocuts += 1
        end

    end
    
    println("Solved in ", k, " iterations.")
    return x, first
end

Let's test it.

In [ ]:
x, firstst = lshaped(scenarios, prob)

The resulting master program is:

In [ ]:
println(firstst)

The optimal value is

In [ ]:
objective_value(firstst)

The optimal solution is

In [ ]:
value.(x)

## Improvements

A more efficient implementation would also to generate a standard second-stage problem and modify only the components affected by the scenario realization.

Alternatively, we could directly implement the dual of the second-stage problems. It is then possible to find a unbounded ray in the situation where we are looking to implement a feasibility cut.

In [ ]:
function secondstage_dual(x, ξ)
    m = Model()

    T = [ sparse([1,2,3,4], [1,2,3,4], [-1,-1,-1,-1]); spzeros(3,4) ]
    h = sparse([ zeros(nFactories); ξ; demands[2:nFlavors]])
    n = length(x)
    
    @variable(m, π[1:nFlavors+nFactories])
    for i = 1:nFactories
        set_upper_bound(π[i], 0)
    end

    for i = nFactories+1:nFlavors+nFactories
        set_lower_bound(π[i], 0)
    end
    
    for i = 1:nFactories
        for j = 1:nFlavors
            @constraint(m, π[i]+π[nFactories+j] <= cost[i,j])
        end
    end
    
    @objective(m, Max, sum((h[i]-dot(T[i,:],x))*π[i] for i = 1:nFactories+nFlavors))

    return m, π, h, T
end

Let's construct the dual of the second-stage program when the first-stage decision is the null vector.

In [ ]:
ξ = 3
x = zeros(4)

m, π, h, t = secondstage_dual(x, ξ)

println(m)

In [ ]:
set_optimizer(m, solver)
optimize!(m)

In other terms, the dual program is unbounded. In order to prevent this situation and obtain a direction that we can exploit, we set arbitrarily large lower and upper bounds.

In [ ]:
function secondstage_dual(x, ξ)
    m = Model()

    T = [ sparse([1,2,3,4], [1,2,3,4], [-1,-1,-1,-1]); spzeros(3,4) ]
    h = sparse([ zeros(nFactories); ξ; demands[2:nFlavors]])
    n = length(x)
    
    @variable(m, y[1:nFlavors+nFactories])
    for i = 1:nFactories
        set_lower_bound(y[i], -1e9)
        set_upper_bound(y[i], 0)
    end

    for i = nFactories+1:nFlavors+nFactories
        set_lower_bound(y[i], 0)
        set_upper_bound(y[i], 1e9)
    end
    
    for i = 1:nFactories
        for j = 1:nFlavors
            @constraint(m, y[i]+y[nFactories+j] <= cost[i,j])
        end
    end
    
    @objective(m, Max, sum((h[i]-dot(T[i,:],x))*y[i] for i = 1:nFactories+nFlavors))

    return m, y, h, T
end

In [ ]:
m, y, h, t = secondstage_dual(x, ξ)
set_optimizer(m, solver)
optimize!(m)

The solution $y$ has components reaching the bounds, and we can normalize it to obtain a direction in which the dual is increasing towards $\infty$.

In [ ]:
value.(y)

We can normalize using the infinite norm.

In [ ]:
value.(y)/norm(value.(y), Inf)